# Bibliotecas

In [1]:
import ranking
import salvar_resultados as sr

import pandas as pd
import tensorflow as tf
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import time

In [2]:
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [3]:
#embed("aaaa")

# Constantes

# Dataset

In [4]:
#arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']
#arquivos = ['ale_1_1']
#arquivos = ['ale_5_1']
#arquivos = ['hn_1_1']
arquivos = ['hn_5_1']

lista_df_treino = []
lista_df_teste = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Datasets/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_teste = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df_treino)
    lista_df_teste.append(df_teste)

## Funções

In [5]:
def calcular_embedding(modelo, texto):

    #embedding = modelo.encode(texto)
    embedding = modelo(texto)

    return embedding


def pipeline_usem(df):
    
    # iniciando o modelo
    modelo = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
    
    # Calculando embeddings
    embedding = df.apply(lambda linha: calcular_embedding(modelo, linha['titulo']), axis = 1)

    return embedding

## Universal Sentence Encoder Multilingual

In [6]:
flag_usem = True
if flag_usem == True:

    dir_metricas = f"Dados/Resultados/Ranqueado/USEM"

    #tam = 10
    for nome, df_teste in zip(arquivos, lista_df_teste):

        #df_teste = df_teste[:tam]

        df_concat, df_matches = ranking.criar_df_teste(df_teste, "titulo_1", "titulo_2", 'titulo')

        inicio_tempo = time.time()
        embedding = pipeline_usem(df_concat)
        resultado = ranking.calcular_dis_2_vetores_cond(embedding, df_matches)

        final_tempo = time.time()

        # adicionando o tempo que demorou para o algoritmo rodar
        ranking.calcular_tempo(df_matches, inicio_tempo, final_tempo)

        # calcular acuracia_k e match rank
        indices, valores = ranking.calcular_metricas(df_matches, df_concat, resultado)

        #salvar métricas
        sr.salvar_resultado(nome, df_matches, dir_metricas)
        sr.salvar_resultado_categoria(nome, df_matches, dir_metricas)

c:\Users\llvs2\anaconda3\envs\tcc_env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [7]:
    a

NameError: name 'a' is not defined

# Resultados